In [28]:
import pandas as pd
import re
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import squarify
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso

In [29]:
#Function สำหรับการเช็คค่าทางสถิติและดูภาพรวมของข้อมูลเพื่อทำความเข้าใจข้อมูลให้มากยิ่งขึ้น
def summarize_dataframe(df):
    summary_data = []

    for column in df.columns:
        data_type = df[column].dtype
        unique_values_count = df[column].nunique()
        value_counts = df[column].value_counts()
        missing_values_count = df[column].isnull().sum()
        
        # คำนวณ Min, Max, Mean และแปลงค่าเป็นเลขฐาน 10
        if pd.api.types.is_numeric_dtype(df[column]):
            min_value = round(df[column].min())
            max_value = round(df[column].max())
            mean_value = round(df[column].mean())
            median_value = round(df[column].median())
            std_value = round(df[column].std())

            # สร้างคอลัมน์ Range
            range_value = f"{min_value:.10g} - {max_value:.10g}"
            
            # แปลง Mean ให้อยู่ในเลขฐาน 10
            mean_value = f"{mean_value:.10g}"
            
            # แปลง Mean ให้อยู่ในเลขฐาน 10
            median_value = f"{median_value:.10g}"
            
            # แปลง Mean ให้อยู่ในเลขฐาน 10
            std_value = f"{std_value:.10g}"

        else:
            range_value = None
            mean_value = None
            median_value = None
            std_value = None
        
        # คำนวณ Outliers
        if pd.api.types.is_numeric_dtype(df[column]):
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            count_lower_outliers = df[df[column] < lower_bound].shape[0]
            count_upper_outliers = df[df[column] > upper_bound].shape[0]
        else:
            count_lower_outliers = None
            count_upper_outliers = None

        # ตัวอย่างข้อมูลในแต่ละค่า
        examples = value_counts.index.tolist()[:5]
        
        # นับจำนวนในแต่ละค่า
        #counts = value_counts.tolist()[:2]
        
        summary_data.append({
            'Attribute': column,
            'Data Type': data_type,
            'Row' : len(df),
            'Unique Values': unique_values_count,
            'Missing Values': missing_values_count,
            'Range (Min - Max)': range_value,
            'Mean': mean_value,
            'Medium' : median_value,
            'STD' : std_value,
            'Lower Outliers Count': count_lower_outliers,
            'Upper Outliers Count': count_upper_outliers,
            'Values': examples,
            #'Counts': counts
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    return summary_df

#ตรวจสอบความถูกต้องและสอดคล้องของข้อมูล จะคืนค่าเป็น records ที่ไม่สอดคล้อง
def get_non_matching_records(df1, attrA, df2, attrB):
    non_matching = df1[~df1[attrA].isin(df2[attrB])]
    
    return non_matching

#หาpattern ข้อมูลค่าใน Attr และหา Error
def detect_patterns(df, column):
    # เก็บรูปแบบที่พบในคอลัมน์
    patterns = []

    for value in df[column].dropna():
        # แปลงค่าทั้งหมดให้เป็นสตริง
        value_str = str(value)
        
        # หารูปแบบทั่วไปโดยการแทนที่ตัวเลขด้วย "D" และตัวอักษรด้วย "A"
        pattern = re.sub(r'\d', 'D', value_str)
        pattern = re.sub(r'[a-zA-Z]', 'A', pattern)
        pattern = re.sub(r'\s+', ' ', pattern)  # ลบช่องว่างที่เกินมา
        
        patterns.append(pattern)
    
    # นับความถี่ของแต่ละรูปแบบ
    pattern_counts = Counter(patterns)
    
    # สร้าง DataFrame ที่มีคอลัมน์ 'patterns' และ 'count'
    pattern_df = pd.DataFrame(pattern_counts.items(), columns=['patterns', 'count'])
    
    return pattern_df


#คืน records ที่มีต่าตรกับ Pattern ที่ใส่เข้าไป
def filter_by_pattern(df, column, pattern):
    # สร้างฟังก์ชันภายในเพื่อแปลงค่าของคอลัมน์เป็นรูปแบบเดียวกันกับ detect_patterns
    def revert_pattern(value):
        value_str = str(value)
        revert_pattern = re.sub(r'\d', 'D', value_str)
        revert_pattern = re.sub(r'[a-zA-Z]', 'A', revert_pattern)
        revert_pattern = re.sub(r'\s+', ' ', revert_pattern)
        return revert_pattern
    
    # กรอง records ที่มี pattern ตรงกับที่กำหนด
    matching_records = df[df[column].apply(revert_pattern) == pattern]
    
    return matching_records


def convert_time_of_transaction(df, column):
    # แปลงค่า time_of_transaction เป็นรูปแบบเวลา HH:MM
    df[column] = df[column].apply(lambda x: f"{int(x // 100):02}:{int(x % 100):02}")
    return df

# Data Preprocessing

In [30]:
#Load Data
causal = pd.read_csv('dh_causal_lookup.csv')
product= pd.read_csv('111.csv')
store = pd.read_csv('dh_store_lookup.csv')
transactions = pd.read_csv('dh_transactions.csv')

In [31]:
causal_summarize = summarize_dataframe(causal)
product_summarize = summarize_dataframe(product)
store_summarize = summarize_dataframe(store)
transactions_summarize = summarize_dataframe(transactions)

In [32]:
#causal_summarize
#product_summarize
#store_summarize
#transactions_summarize

In [33]:
# ทำการ Merge โดยจับคู่คีย์ทั้ง 5 ได้แก่ upc, week, geography, store, household
df_merged = pd.merge(transactions, causal[['upc', 'week', 'geography', 'store', 'display_desc']],
                     on=['upc', 'week', 'geography', 'store'], how='left')

df_merged = df_merged.dropna() #ตัดทุก records ที่เป็น Null ออก ซึ่งมันจะเป็น records ที่สินค้าหรือร้านหรือวีคไม่มีแสดงการจัดวาง
df_merged

,upc,dollar_sales,units,time_of_transaction,geography,week,household,store,basket,day,coupon,display_desc
1502430,9999985046,0.79,1,1125,2,43,124468,244,964711,300,0,Not on Display
1502431,9999985040,0.79,1,1125,2,43,124468,244,964711,300,0,Not on Display
1502441,9999985047,1.58,2,2138,2,43,18425,244,964718,300,0,Not on Display
1502444,2700042254,0.89,1,1838,2,43,389596,365,964720,300,0,In-Aisle
1502445,9999967727,1.29,1,1911,2,43,148063,365,964721,300,0,Secondary Location Display
...,...,...,...,...,...,...,...,...,...,...,...,...
5197629,3620000473,2.00,1,2334,1,104,379026,108,3316316,728,0,Not on Display
5197640,3620000473,2.00,1,2234,1,104,174208,21,3316320,728,0,Not on Display
5197642,3620000432,2.00,1,2234,1,104,174208,21,3316320,728,0,Not on Display
5197643,3620000439,2.00,1,2234,1,104,174208,21,3316320,728,0,Not on Display


#### feature engineering

In [91]:
# รวมข้อมูลตาม upc, week, geography, store และหาผลรวมของ dollar_sales และ units
transactions_combine_by_upc_store_week = df_merged.groupby(['upc', 'week', 'geography', 'store','display_desc']).agg(
        total_dollar_sales=('dollar_sales', 'sum'),    
        total_units=('units', 'sum')                   
    ).reset_index()
transactions_combine_by_upc_store_week = transactions_combine_by_upc_store_week.sort_values(by=['upc', 'week'])
transactions_combine_by_upc_store_week.sort_values(by=['total_units'])

final_transactions_combine = transactions_combine_by_upc_store_week
final_transactions_combine

,upc,week,geography,store,display_desc,total_dollar_sales,total_units
0,601011292,44,1,159,Secondary Location Display,2.00,1
1,601011292,45,1,159,Secondary Location Display,10.00,5
2,601011292,46,2,326,In-Aisle,5.00,2
3,601011292,47,2,326,Rear End Cap,2.50,1
4,601011292,49,1,1,Not on Display,2.00,1
...,...,...,...,...,...,...,...
173896,9999985261,104,1,154,Secondary Location Display,8.10,6
173897,9999985261,104,1,170,Secondary Location Display,3.87,3
173898,9999985261,104,1,171,Side-Aisle End Cap,6.75,5
173899,9999985766,58,1,205,In-Shelf,1.25,1


In [92]:
#final_transactions_combine_summarize = summarize_dataframe(final_transactions_combine)
#final_transactions_combine_summarize

In [93]:
#ตัด Outlier
Q1 = final_transactions_combine['total_dollar_sales'].quantile(0.25)
Q3 = final_transactions_combine['total_dollar_sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


final_transactions_combine = final_transactions_combine[(final_transactions_combine['total_dollar_sales'] >= lower_bound) & 
                                                        (final_transactions_combine['total_dollar_sales'] <= upper_bound)]
final_transactions_combine

,upc,week,geography,store,display_desc,total_dollar_sales,total_units
0,601011292,44,1,159,Secondary Location Display,2.00,1
2,601011292,46,2,326,In-Aisle,5.00,2
3,601011292,47,2,326,Rear End Cap,2.50,1
4,601011292,49,1,1,Not on Display,2.00,1
5,601011292,49,1,3,Not on Display,4.00,2
...,...,...,...,...,...,...,...
173895,9999985261,104,1,126,Rear End Cap,1.35,1
173897,9999985261,104,1,170,Secondary Location Display,3.87,3
173898,9999985261,104,1,171,Side-Aisle End Cap,6.75,5
173899,9999985766,58,1,205,In-Shelf,1.25,1


In [94]:
#final_transactions_combine_summarize = summarize_dataframe(final_transactions_combine)
#final_transactions_combine_summarize

In [95]:
final_transactions_combine = final_transactions_combine.loc[(final_transactions_combine['store'] == 270)]
#final_transactions_combine = final_transactions_combine.loc[(final_transactions_combine['upc'] == 601011292)]
df = final_transactions_combine.sort_values(by=['week'])  
df

,upc,week,geography,store,display_desc,total_dollar_sales,total_units
158474,9999985046,43,2,270,Not on Display,0.79,1
37022,3620000444,43,2,270,Mid-Aisle End Cap,6.15,5
160048,9999985050,43,2,270,Not on Display,2.37,3
69961,3620004155,43,2,270,Secondary Location Display,5.98,2
159259,9999985048,43,2,270,Not on Display,3.16,4
...,...,...,...,...,...,...,...
60017,3620001365,104,2,270,Not on Display,4.29,3
170038,9999985165,104,2,270,Mid-Aisle End Cap,4.83,7
93582,5100002794,104,2,270,In-Shelf,5.00,3
49124,3620000473,104,2,270,Not on Display,6.00,3


In [96]:
#final_transactions_combine_s =  summarize_dataframe(final_transactions_combine)
#final_transactions_combine_s

In [97]:
#เปลี่ยนกับไม่เปลี่ยน type ผลใกล้เคียงกัน แค่อันนี้นานกว่า
#final_transactions_combine['upc'] = final_transactions_combine['upc'].astype(str)

In [98]:
df = final_transactions_combine

# encoded display_desc ให้เป็น cat
df['display_desc_encoded'] = df['display_desc'].astype('category').cat.codes


X = df[['upc', 'week', 'geography', 'store', 'display_desc_encoded']]
#y = df['total_units']
y = df['total_dollar_sales']

# แบ่งชุดข้อมูลตามลำดับเวลา (ใช้ TimeSeriesSplit)
split = TimeSeriesSplit(n_splits=3)

for train_index, test_index in split.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # แบ่ง validation set จาก train set
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

/var/folders/hj/vmbgf9dd4tq0dvgjdngp58kw0000gn/T/ipykernel_3568/2826179796.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_desc_encoded'] = df['display_desc'].astype('category').cat.codes


# Model Selection

In [99]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

rf_mse, gb_mse, poisson_mse, linear_mse, dt_mse = [], [], [], [], []
rf_mae, gb_mae, poisson_mae, linear_mae, dt_mae = [], [], [], [], []

for train_index, val_index in split.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    #Random Forest
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_pred = np.round(rf_model.predict(X_val))  # ปัดเศษให้เป็นจำนวนเต็ม
    rf_mse.append(mean_squared_error(y_val, rf_pred))
    rf_mae.append(mean_absolute_error(y_val, rf_pred))

    #Gradient Boosting
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_model.fit(X_train, y_train)
    gb_pred = np.round(gb_model.predict(X_val))  # ปัดเศษให้เป็นจำนวนเต็ม
    gb_mse.append(mean_squared_error(y_val, gb_pred))
    gb_mae.append(mean_absolute_error(y_val, gb_pred))

    #Poisson Regression
    poisson_model = PoissonRegressor()
    poisson_model.fit(X_train, y_train)
    poisson_pred = np.round(poisson_model.predict(X_val))  # ปัดเศษให้เป็นจำนวนเต็ม
    poisson_mse.append(mean_squared_error(y_val, poisson_pred))
    poisson_mae.append(mean_absolute_error(y_val, poisson_pred))

    #Linear Regression
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    linear_pred = np.round(linear_model.predict(X_val))  # ปัดเศษให้เป็นจำนวนเต็ม
    linear_mse.append(mean_squared_error(y_val, linear_pred))
    linear_mae.append(mean_absolute_error(y_val, linear_pred))

    #Decision Tree Regressor
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train, y_train)
    dt_pred = np.round(dt_model.predict(X_val))  # ปัดเศษให้เป็นจำนวนเต็ม
    dt_mse.append(mean_squared_error(y_val, dt_pred))
    dt_mae.append(mean_absolute_error(y_val, dt_pred))

print("Random Forest MSE:", np.mean(rf_mse), "MAE:", np.mean(rf_mae))
print("Gradient Boosting MSE:", np.mean(gb_mse), "MAE:", np.mean(gb_mae))
print("Poisson Regression MSE:", np.mean(poisson_mse), "MAE:", np.mean(poisson_mae))
print("Linear Regression MSE:", np.mean(linear_mse), "MAE:", np.mean(linear_mae))
print("Decision Tree MSE:", np.mean(dt_mse), "MAE:", np.mean(dt_mae))


/Users/lapislux/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:295: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights


Random Forest MSE: 3.6172462009803925 MAE: 1.5697426470588234
Gradient Boosting MSE: 3.555285416666667 MAE: 1.5431249999999999
Poisson Regression MSE: 3.2253099264705885 MAE: 1.4883946078431372
Linear Regression MSE: 3.331412867647059 MAE: 1.5115073529411764
Decision Tree MSE: 4.687172671568628 MAE: 1.7196691176470589


#  Model Trianing

In [100]:
#จริงๆ Poisson Regression ให้ดีสุดแต่มันจะคืนค่ามาเฉพาะตำนวนเต็ม และก็ใช้เวลานานมาก ตอนแรกเอาไปใช้เพื่อทำนาย total_units มันจะดีสุด
#เลยเลือกใช้ Linear Regression แทน
#หา Best parameters/Hyperparameters แต่มีตัวเดียวที่เหลือถ้าไม่เกี่ยวก็มีแตทำให้แย่ลง
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

# Ridge Regression
ridge_grid = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(X_train, y_train)
ridge_best = ridge_grid.best_estimator_
print('Best Ridge Alpha:', ridge_grid.best_params_)
print('Best Ridge MSE:', -ridge_grid.best_score_)

# Lasso Regression
lasso_grid = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(X_train, y_train)
lasso_best = lasso_grid.best_estimator_
print('Best Lasso Alpha:', lasso_grid.best_params_)
print('Best Lasso MSE:', -lasso_grid.best_score_)


Best Ridge Alpha: {'alpha': 100.0}
Best Ridge MSE: 3.218564494963185
Best Lasso Alpha: {'alpha': 100.0}
Best Lasso MSE: 3.2054820139025715


/Users/lapislux/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.35053e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/lapislux/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.6232e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/lapislux/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.58493e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/lapislux/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.54857e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", o

# Model Evaluation

In [101]:
from sklearn.model_selection import cross_val_score

# ใช้ Cross-Validation บน Linear Regression
cv_mse = cross_val_score(LinearRegression(), X, y, cv=5, scoring='neg_mean_squared_error')
cv_mae = cross_val_score(LinearRegression(), X, y, cv=5, scoring='neg_mean_absolute_error')

print("Cross-Validation MSE (Linear Regression):", -np.mean(cv_mse))
print("Cross-Validation MAE (Linear Regression):", -np.mean(cv_mae))


Cross-Validation MSE (Linear Regression): 3.2332347712430347
Cross-Validation MAE (Linear Regression): 1.5187510659400858


# Prediction

In [109]:
import joblib

joblib.dump(linear_model, 'linear_model.pkl')

model = joblib.load('linear_model.pkl')

In [106]:
best_model = ridge_best 
y_pred = best_model.predict(X_test)
result_df = pd.DataFrame(X_test, columns=X.columns)
result_df['Y_true'] = y_test.values  
result_df['Y_predict'] = y_pred  
result_df.sort_values(by=['upc']) 

,upc,week,geography,store,display_desc_encoded,Y_true,Y_predict
129316,9999967586,72,2,270,3,1.50,2.749639
129422,9999967586,73,2,270,2,3.00,2.827625
129478,9999967586,74,2,270,2,7.50,2.823340
129640,9999967648,47,2,270,0,2.58,3.103589
129651,9999967648,49,2,270,5,1.29,2.683662
...,...,...,...,...,...,...,...
172155,9999985261,65,2,270,2,3.96,2.861907
172001,9999985261,51,2,270,5,1.29,2.675089
173754,9999985261,93,2,270,1,3.30,2.824186
173055,9999985261,84,2,270,5,3.70,2.533670
